### Model API Workflow

1. CloudWatch Events로부터 호출 받을 때, 시간을 보내거나, image 내에서 현재 시간 얻어내거나. (난이도로 봤을 때 훨씬 쉬운건 아무것도 안보내고, image 내에서 진행하는 것.)

2. 시간을 20분 단위로 내림하기.

3. RDS로부터 내림한 시간을 기준으로 데이터 받음. (evs info & timestamp's sequence(length: n) & pymysql)

4. 받은 데이터 utils.py 돌리고 model 실행.

5. RDS로부터 데이터 전달하여 update.

### 내림 시간 계산

In [154]:
import datetime

def get_dt():
    now = datetime.datetime.now()
    dt = datetime.datetime(now.year, now.month, now.day, now.hour, now.minute//20*20, 00)
    str_dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    return dt, str_dt

get_dt()

(datetime.datetime(2023, 8, 14, 16, 40), '2023-08-14 16:40:00')

### DB 전, 후 Datetime 계산 (sequence)

In [169]:
def get_in_dt(n):
    in_dt = []
    dt, _ = get_dt()
    for i in range(n):
        if i == 0 or i == n-1:
            idt = dt - datetime.timedelta(minutes = 20*(i+1))
            in_dt.append(idt.strftime('%Y-%m-%d %H:%M:%S'))
    return in_dt

get_in_dt(12)

['2023-08-14 16:20:00', '2023-08-14 12:40:00']

In [170]:
def get_out_dt(n):
    out_dt = []
    dt, _ = get_dt()
    for i in range(n):
        if i == 0 or i == n-1:
            odt = dt + datetime.timedelta(minutes = 20*(i+1))
            out_dt.append(odt.strftime('%Y-%m-%d %H:%M:%S'))
    return out_dt

get_out_dt(6)

['2023-08-14 17:00:00', '2023-08-14 18:40:00']

### DB 연동 및 데이터 입력

In [127]:
import pymysql
import pandas as pd

attributes = pd.read_csv('data/evcs_attributes.csv')
embedding = pd.read_csv('data/evcs_embedding.csv')

In [128]:
def db_info():
    host = "db-hwangpeng.c753snssvybo.ap-northeast-2.rds.amazonaws.com"
    user = "seokyang"
    password = "hwangpeng"
    db = "evs"

    return host, user, password, db

host, user, password, db = db_info()

connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

In [129]:
# Create Table
create_query = '''
CREATE TABLE IF NOT EXISTS station (
Sid INT NOT NULL PRIMARY KEY, 
Sname VARCHAR(100) NOT NULL, 
Latitude DOUBLE NOT NULL, 
Longitude DOUBLE NOT NULL, 
Capacity FLOAT NOT NULL, 
Slow_Chargers INT NOT NULL,
Fast_Chargers INT NOT NULL,
Mean_trip DOUBLE NOT NULL,
Length_city DOUBLE NOT NULL,
Length_highway DOUBLE NOT NULL,
Length_local DOUBLE NOT NULL,
Length_national DOUBLE NOT NULL,
Indust_ratio DOUBLE NOT NULL,
Etc_ratio DOUBLE NOT NULL,
Green_ratio DOUBLE NOT NULL,
Commerce_ratio DOUBLE NOT NULL,
Reside_ratio DOUBLE NOT NULL,
UMAP_1 DOUBLE NOT NULL,
UMAP_2 DOUBLE NOT NULL,
UMAP_3 DOUBLE NOT NULL,
UMAP_4 DOUBLE NOT NULL,
UMAP_5 DOUBLE NOT NULL,
UMAP_6 DOUBLE NOT NULL,
UMAP_7 DOUBLE NOT NULL,
UMAP_8 DOUBLE NOT NULL
)
'''

with connection.cursor() as cursor:
    cursor.execute(create_query.replace('\n', ''))

In [130]:
# Insert Values into Database
for idx in range(attributes.shape[0]):
    atts = attributes.iloc[idx, :].values.tolist()
    emb = embedding.iloc[idx, :].values.tolist()[1:]
    values = tuple([idx] + atts + emb)
    insert_query = "INSERT INTO station VALUES {};"
    
    with connection.cursor() as cursor:
        cursor.execute(insert_query.format(values))

connection.commit()
connection.close()

### DB 가져오기

In [161]:
import pymysql
import pandas as pd

In [162]:
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

select_query = "SELECT * FROM sequence WHERE Time = '2022-03-20 00:00:00'"

with connection.cursor() as cursor:
    cursor.execute(select_query)
    result = cursor.fetchall()

In [163]:
result

((0, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (2, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (3, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (4, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (5, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (6, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (9, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (10, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (11, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (13, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (14, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (16, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (19, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (20, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (21, datetime.datetime(2022, 3, 20, 0, 0), 1),
 (22, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (23, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (24, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (25, datetime.datetime(2022, 3, 20, 0, 0), 1),
 (26, datetime.datetime(2022, 3, 20, 0, 0), 0),
 (27, datetime.datetime(2022, 3, 20, 0, 0), 0),

In [137]:
atts_cols = ['Latitude', 'Longitude', 'Capacity', 'Slow_Chargers',
       'Fast_Chargers', 'Mean_trip', 'Length_city', 'Length_highway',
       'Length_local', 'Length_national', 'Indust_ratio', 'Etc_ratio',
       'Green_ratio', 'Commerce_ratio', 'Reside_ratio']

embed_cols = ['UMAP_1', 'UMAP_2', 'UMAP_3', 
        'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7', 'UMAP_8']

In [5]:
df = pd.DataFrame(result, columns=[col[0] for col in cursor.description])

atts = df.loc[:, atts_cols]
embed = df.loc[:, embed_cols]

In [6]:
from utils import json2tensor
Sids, X = json2tensor(atts.to_dict('index'))

In [7]:
X

tensor([[1.2891e+02, 3.7744e+01, 5.0000e+01,  ..., 4.3682e-02, 0.0000e+00,
         9.5632e-01],
        [1.2890e+02, 3.7755e+01, 5.0000e+01,  ..., 2.6380e-01, 3.2426e-01,
         4.1194e-01],
        [1.2891e+02, 3.7754e+01, 5.0000e+01,  ..., 2.7188e-01, 0.0000e+00,
         7.2812e-01],
        ...,
        [1.2792e+02, 3.7343e+01, 5.0000e+01,  ..., 7.2788e-01, 1.5569e-01,
         1.1643e-01],
        [1.2934e+02, 3.6019e+01, 5.0000e+01,  ..., 3.3261e-01, 1.2972e-02,
         6.5442e-01],
        [1.2773e+02, 3.5521e+01, 5.0000e+01,  ..., 1.2440e-01, 1.8324e-01,
         6.9236e-01]])

### DB 입력

In [2]:
def update_occupancy(y_json):

    connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

    create_query = '''
CREATE TABLE occupancy (
Sid INT NOT NULL PRIMARY KEY, 
Occupancy_20 INT NOT NULL,
Occupancy_60 INT NOT NULL,
Occupancy_120 INT NOT NULL,
FOREIGN KEY (sid) REFERENCES station(sid)
)
'''
    select_query = "SELECT sid FROM station"

    insert_query = "INSERT INTO station VALUES {};"

    update_query = "UPDATE occupancy SET Occupancy_20 = %d Occupancy_60 = %d Occupancy_120 = %d WHERE Sid = %d"

    try:
        with connection.cursor() as cursor:
                cursor.execute(create_query.replace('\n', ''))
                cursor.execute(select_query)
                result = cursor.fetchall()

        insert_query = "INSERT INTO occupancy VALUES {};"
        for idx in result:
            values = tuple(list(idx) + y_json[idx[0]])
            with connection.cursor() as cursor:
                cursor.execute(insert_query.format(values))
            
        connection.commit()
        connection.close()

    except:
        with connection.curosr() as cursor:
            cursor.execute(update_query, tuple(y_json[idx[0]], list(idx)))


### Keep in mind
- model, input이 제대로 주어지면 utils.py, inference.py 수정
- 현재 시간 기준으로 가져올 것이기 때문에 Table에 데이터 입력시 2023년 8~10월로 맞춰야함.

In [3]:
import pandas as pd

df = pd.read_csv('data/evcs_attributes.csv')

In [7]:
df.set_index('sid')

,latitude,longitude,capacity,slow,fast,mean_trip,length_city,length_highway,length_local,length_national,indust_ratio,etc_ratio,green_ratio,commerce_ratio,reside_ratio
sid,,,,,,,,,,,,,,,
강원_강릉_강남동주민센터,128.905083,37.744127,50.000000,0.0,1.0,298.558333,10446.126,0.0,0.000,0.000,0.000000,0.000000,0.043682,0.000000,0.956318
강원_강릉_강릉농협 하나로마트,128.901915,37.754689,50.000000,0.0,1.0,1896.051515,13062.524,0.0,0.000,796.517,0.000000,0.000000,0.263801,0.324259,0.411940
강원_강릉_강릉축협 하나로마트,128.912640,37.754163,50.000000,0.0,1.0,742.025000,10648.559,0.0,0.000,0.000,0.000000,0.000000,0.271876,0.000000,0.728124
강원_강릉_강원도교육연수원,128.918339,37.786442,50.000000,0.0,1.0,1127.221053,10117.694,0.0,0.000,0.000,0.000000,0.000000,0.580970,0.000000,0.419030
강원_강릉_강원도립대학,128.829369,37.877095,50.000000,0.0,1.0,341.908333,3863.825,0.0,0.000,1830.095,0.000131,0.000000,0.591334,0.000000,0.408535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
환경부_환경부_제주공항,126.493376,33.504757,43.307207,6.0,9.0,1540.785714,5261.158,0.0,0.000,0.000,0.000000,0.000000,1.000000,0.000000,0.000000
환경부_환경부_진영공설운동장,128.718358,35.307782,50.000000,0.0,1.0,172.346269,8081.601,0.0,0.000,3000.002,0.000000,0.423613,0.357025,0.000000,0.219362
환경부_환경부_차량등록사업소,127.923780,37.343286,50.000000,0.0,1.0,892.230612,11386.767,0.0,0.000,0.000,0.000000,0.000000,0.727875,0.155694,0.116431


In [40]:
df = pd.read_csv('data/inference_data_220320sun_200326satur.csv')
df.head()

,time,강원_강릉_HAPPY700 평창시네마,강원_강릉_감자연구소,강원_강릉_강남동주민센터,강원_강릉_강릉축협 하나로마트,강원_강릉_강원도교육연수원,강원_강릉_강원도립대학,강원_강릉_강원양돈농협 하나로마트,강원_강릉_경포해수욕장,강원_강릉_교1동주민센터,...,충북_충북직할_충청북도보건환경연구원,충북_충주_노은면 행정복지센터,충북_충주_성서동 공영주차장,충북_충주_수안보 증원회관,충북_충주_이마트 충주점,충북_충주_중앙탑면 행정복지센터,충북_충주_충주시청 민원주차장,충북_충주_충주실내체육관,충북_충주_충주종합스포츠타운,충북_충주_평생학습관
0,2022-03-20 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-03-20 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-03-20 00:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-03-20 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-03-20 01:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df.shape # shape -> (504*944, 3)
# df.stack().reset_index()

(504, 944)

In [75]:
occupancy = df.set_index('time').stack().reset_index()
occupancy.columns = ['time', 'sid', 'occupancy']
occupancy

,time,sid,occupancy
0,2022-03-20 00:00:00,강원_강릉_HAPPY700 평창시네마,0.0
1,2022-03-20 00:00:00,강원_강릉_감자연구소,0.0
2,2022-03-20 00:00:00,강원_강릉_강남동주민센터,0.0
3,2022-03-20 00:00:00,강원_강릉_강릉축협 하나로마트,0.0
4,2022-03-20 00:00:00,강원_강릉_강원도교육연수원,0.0
...,...,...,...
475267,2022-03-26 23:40:00,충북_충주_중앙탑면 행정복지센터,0.0
475268,2022-03-26 23:40:00,충북_충주_충주시청 민원주차장,0.0
475269,2022-03-26 23:40:00,충북_충주_충주실내체육관,0.0
475270,2022-03-26 23:40:00,충북_충주_충주종합스포츠타운,0.0


In [109]:
attr = pd.read_csv('data/evcs_attributes.csv')
df = pd.merge(occupancy, attr.reset_index().rename(columns={'index': 'id'})[['id', 'sid']], on = 'sid', how = 'left').dropna()

In [111]:
df.to_csv('data/evcs_sequence.csv', index=False)

In [125]:
sequence = pd.read_csv('data/evcs_sequence.csv')

create_query = '''
CREATE TABLE IF NOT EXISTS sequence (
Sid INT NOT NULL, 
Time DATETIME NOT NULL,
Occupancy INT NOT NULL,
PRIMARY KEY (Sid, Time)
)
'''
import pymysql
from private import db_info

host, user, password, db = db_info()

connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    db=db,
    charset='utf8',
    port = 3306
)

with connection.cursor() as cursor:
    cursor.execute(create_query.replace('\n', ''))

for idx, row in sequence.iterrows():
    elements = row.values.tolist()
    values = tuple([elements[-1]] + [elements[0]] + [elements[-2]])

    insert_query = "INSERT INTO sequence VALUES {};"
    
    with connection.cursor() as cursor:
        cursor.execute(insert_query.format(values))

KeyboardInterrupt: 

In [88]:
occupancy.pivot(index='time', columns='sid', values='occupancy').reset_index()

sid,time,강원_강릉_HAPPY700 평창시네마,강원_강릉_감자연구소,강원_강릉_강남동주민센터,강원_강릉_강릉축협 하나로마트,강원_강릉_강원도교육연수원,강원_강릉_강원도립대학,강원_강릉_강원양돈농협 하나로마트,강원_강릉_경포해수욕장,강원_강릉_교1동주민센터,...,충북_충북직할_충청북도보건환경연구원,충북_충주_노은면 행정복지센터,충북_충주_성서동 공영주차장,충북_충주_수안보 증원회관,충북_충주_이마트 충주점,충북_충주_중앙탑면 행정복지센터,충북_충주_충주시청 민원주차장,충북_충주_충주실내체육관,충북_충주_충주종합스포츠타운,충북_충주_평생학습관
0,2022-03-20 00:00:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-03-20 00:20:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-03-20 00:40:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-03-20 01:00:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-03-20 01:20:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2022-03-26 22:20:00,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500,2022-03-26 22:40:00,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
501,2022-03-26 23:00:00,0.0,0.0,0.045833,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,2022-03-26 23:20:00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
import pandas as pd

pd.read_csv('data/evcs_sequence.csv')['time'][0]

'2022-03-20 00:00:00'